<center><h1><b>Write a script add data to a Elastic Search based Backend database.</b></h1></center>

<p> I just make a copy of <b>Excel sheet name: beginner_assignment01</b> save to my google drive to perform required task. Required Library to be install:
<li>1. pip install gspread</li>
    <li>2. pip install oauth2client</li>
    <li>3. pip install mysql-connector</li>
    <li>4. pip install elasticsearch</li>
Now Follow these steps to access Google Sheet via python :
    <li>1. Go to the <b>Google APIs Console</b>.</li>
    <li>2. Create a <b>new Project</b>.</li>
    <li>3. Click <b>Enable API</b>. Search for and enable <b>Google Drive API</b>.</li>
    <li>4. <b>Create Credentials</b> and Select <b>WebServer</b> and <b>Application Data</b>. In Project Role Select <b>Editor</b>.</li>
<li>5. Download JSON file save to your project folder and rename it as your required.</li>
    <li><b>Open the JSON file and copy the cilent_email address and Share your Google Sheet with that Email ID with Write Permisssion. Otherwise you will get error SheetsNotFound</b></li>
    <b>Note: When you run the script first time it gives error as to enable Google Sheet just click the link and its done. </b>
</p>

In [1]:
# import libraries
import gspread
import mysql_credential as key  # Your MySQl Credential python file here: mysql_credential.py
import mysql.connector
from oauth2client.service_account import ServiceAccountCredentials
# Install the elastic search library to use with python
from elasticsearch import Elasticsearch

In [2]:
# initialize variables for gspread
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('secret.json', scope)
client = gspread.authorize(creds)

In [3]:
# define method to pull data from spreadsheet
def GetSpreadsheetData(sheetName, worksheetIndex):    
    sheet = client.open(sheetName).get_worksheet(worksheetIndex)    
    return sheet.get_all_values()[1:]

In [4]:
# Client Email inside secret.json file must have edit permission on the Google Sheet you are using
data_product = GetSpreadsheetData('Copy of beginner_assignment01', 0)

In [5]:
# View Data From Google Sheets 
print(data_product[0])
print(len(data_product))

['Demo Product 1', 'Product Model 1', '98765987659801', 'Group 1', '1001']
110


In [6]:
# Client Email inside secret.json file must have edit permission on the Google Sheet you are using
data_group = GetSpreadsheetData('Copy of beginner_assignment01', 1)

In [7]:
# View Data From Google Sheets
data_group
print(data_group[1][0])
print(len(data_group))

Group 2
22


In [8]:
#connect to our cluster
es = Elasticsearch([{'host': '101.53.136.181', 'port': 9200}])

In [9]:
# Delete Elastic Search Index
es.indices.delete(index='grouplisting', ignore=[400, 404])

{'acknowledged': True}

In [10]:
# Delete Elastic Search Index
es.indices.delete(index='productlisting', ignore=[400, 404])

{'acknowledged': True}

In [11]:
# Writing Data to Elastic Search For Group Listing
for i in range(len(data_group)):
    es.index(index='grouplisting', doc_type='group', id=i+1, body={
        "group name": data_group[i][0],
        "group description": data_group[i][1],
        "isActive": data_group[i][2],
    })

In [12]:
# Writing Data to Elastic Search For Product Listing
for i in range(len(data_product)):
    es.index(index='productlisting', doc_type='product', id=i+1, body={
        "Product Name": data_product[i][0],
        "Model Name": data_product[i][1],
        "Product Serial No": data_product[i][2],
        "Group Associated": data_product[i][3],
        "product MRP (rs.)": data_product[i][4],
        
    })